In [ ]:
# Cell 1: Setup (no network calls)
import os
import sys
# Ensure Kaggle dataset path for files uploaded as a dataset
sys.path.insert(0, '/kaggle/input/for_kaggle')
from pathlib import Path
import joblib
import pandas as pd
import numpy as np

print('Python:', sys.version)
print('CWD:', Path.cwd())

# Try to add repository code to path if present in working dir or parent dirs
added = False
p = Path.cwd()
for _ in range(5):
    if (p / 'scripts' / 'load_data.py').exists():
        sys.path.insert(0, str(p))
        added = True
        repo_root = p
        break
    p = p.parent
if not added and Path('/kaggle/input').exists():
    # common Kaggle input location: try to find a dataset folder that looks like this repo
    candidates = list(Path('/kaggle/input').glob('*nfl*'))
    if candidates:
        repo_root = candidates[0]
        sys.path.insert(0, str(repo_root))
        added = True

print('Repo root added to sys.path:' , added)

# Import local helpers (these are safe: they do not perform network calls)
try:
    from scripts.load_data import load_test, load_test_input
    from features import transform_for_inference, add_time_lag_features
except Exception as e:
    print('Failed to import local helpers:', e)
    print('Ensure this notebook has the project files (scripts/, features.py, models/) available in the working dir or uploaded as a Kaggle dataset.')
    raise

In [ ]:
# Cell 2: Load saved model (joblib)
model_path = Path('models') / 'best_model.pkl'
# fallback to common Kaggle dataset path if model not in working dir
if not model_path.exists() and Path('/kaggle/input').exists():
    candidates = list(Path('/kaggle/input').glob('*nfl*'))
    if candidates:
        f = candidates[0] / 'models' / 'best_model.pkl'
        if f.exists():
            model_path = f

print('Loading model from', model_path)
if not model_path.exists():
    raise FileNotFoundError(f'Model not found at {model_path}. Upload `models/best_model.pkl` to the notebook working directory or attach as dataset.')

meta = joblib.load(model_path)
feat_cols = meta['feature_columns']
mx = meta['models']['x']
my = meta['models']['y']
print('Loaded model with', len(feat_cols), 'features')

In [ ]:
# Cell 3: Load test files (local or /kaggle/input fallback)
def _load_file_local_or_input(name):
    p = Path(name)
    if p.exists():
        return pd.read_csv(p)
    if Path('/kaggle/input').exists():
        candidates = list(Path('/kaggle/input').glob('*nfl*'))
        if candidates:
            q = candidates[0] / name
            if q.exists():
                return pd.read_csv(q)
    return pd.DataFrame()

print('Loading test_input.csv and test.csv...')
test_input = _load_file_local_or_input('test_input.csv')
test = _load_file_local_or_input('test.csv')

print('test_input rows:', len(test_input))
print('test rows:', len(test))
if test.empty or test_input.empty:
    raise FileNotFoundError('test.csv or test_input.csv not found. Upload them to the notebook working directory or attach as dataset.')

In [ ]:
# Cell 4: Prepare test features and predict
# Merge and add time-lag features like in training
df = pd.merge(test, test_input, on=['game_id','play_id','nfl_id','frame_id'], how='left', suffixes=(None,'_in'))
df = add_time_lag_features(df)
X_pred = transform_for_inference(df, feat_cols, meta.get('player_position_values', None))
print('Prepared feature matrix shape:', X_pred.shape)

# Make predictions
px = mx.predict(X_pred)
py = my.predict(X_pred)
print('Predictions made:', len(px))

In [ ]:
# Cell 5: Save submission CSV
submission = pd.DataFrame({'x': px, 'y': py})
submission_path = Path('submission_best_model_OFFICIAL.csv')
submission.to_csv(submission_path, index=False)
print('Saved submission to', submission_path, 'rows:', len(submission))

# Quick validation
assert not submission.isna().any().any(), 'Submission contains NaNs'
assert np.isfinite(submission.values).all(), 'Submission contains non-finite values'
print('Submission validation passed')

### Notes
- Run this notebook with Internet disabled in Kaggle Notebook Settings before executing.
- If you need me to attach the published notebook to a submission, run it with Internet disabled, publish the executed version, then upload the CSV and choose this published notebook in the Kaggle submission UI.
- If any cell errors, copy the cell number (1..5) and the full traceback and I will debug it for you.